<a href="https://colab.research.google.com/github/RautRitesh/langgraph/blob/main/Lanngraph_corrective_rag_self_rag_and_agentic_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## In this notebook we will try to create the production ready different RAG approaches that langgraph supports

### We will start with creating a corrective rag which will grade the document and generate the best answer

In [38]:
pip install langchain langgraph langchain-core langchain-community langchain-groq langchain faiss-cpu langchain-huggingface

In [64]:
# let us understand how the document grader works first by creating the retriever using faiss

from langchain_community.document_loaders import TextLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [65]:
loader=TextLoader("/content/nepal_insurance_details.txt")
docs=loader.load()

In [66]:
embeddings=HuggingFaceEmbeddings(model="sentence-transformers/all-MiniLM-L6-v2")

In [67]:
text_splitter=RecursiveCharacterTextSplitter(chunk_overlap=200, chunk_size=800,separators=["\n\n","\n"])
chunk=text_splitter.split_documents(docs)
vector_stores=FAISS.from_documents(chunk, embeddings)

In [68]:
retriever=vector_stores.as_retriever(search_kwargs={"k":4})

In [69]:
result=retriever.invoke("What is the document about?")

In [70]:
print(result)

[Document(id='9b9faabb-8c82-4cdc-b981-a9dfad993254', metadata={'source': '/content/nepal_insurance_details.txt'}, page_content='Enrollment Procedure\nCitizens enroll via local Enrollment Assistants (volunteers) at ward offices, health posts, or empaneled facilities using the Insurance Management Information System (IMIS). Provide citizenship certificate, photos, and pay NPR 3,500 annually for a standard family (recent figure); receive a health insurance card instantly or shortly after.'), Document(id='0ee19dc2-db7f-46f7-b160-bcb243f40aa0', metadata={'source': '/content/nepal_insurance_details.txt'}, page_content='Description of the tests provided free of charge by specific government programs (outside of the standard Health Insurance Board coverage) are:'), Document(id='6d142804-d67e-424b-bb23-6aa3c8f22162', metadata={'source': '/content/nepal_insurance_details.txt'}, page_content='Pay Co-Pay: Cover 20% of approved costs immediately (e.g., NPR 200 on a NPR 1,000 bill); no upfront payme

In [71]:
for doc in result:
  print(doc.id)
  print(doc.metadata)
  print(doc.page_content)
  print("\n\n-------")

9b9faabb-8c82-4cdc-b981-a9dfad993254
{'source': '/content/nepal_insurance_details.txt'}
Enrollment Procedure
Citizens enroll via local Enrollment Assistants (volunteers) at ward offices, health posts, or empaneled facilities using the Insurance Management Information System (IMIS). Provide citizenship certificate, photos, and pay NPR 3,500 annually for a standard family (recent figure); receive a health insurance card instantly or shortly after.


-------
0ee19dc2-db7f-46f7-b160-bcb243f40aa0
{'source': '/content/nepal_insurance_details.txt'}
Description of the tests provided free of charge by specific government programs (outside of the standard Health Insurance Board coverage) are:


-------
6d142804-d67e-424b-bb23-6aa3c8f22162
{'source': '/content/nepal_insurance_details.txt'}
Pay Co-Pay: Cover 20% of approved costs immediately (e.g., NPR 200 on a NPR 1,000 bill); no upfront payment for fully covered vulnerable groups like seniors over 70.
Get Treatment: Receive services; hospital bi

## This is the data that we get from the retriever, now we will use this data to check if it is correct or not than only we will go with the generation ,hence this is the implementation of the corrective rag

# Using and settin up the data now

In [72]:
from langchain_groq import ChatGroq
from langgraph.graph import MessagesState, StateGraph , START , END
from langgraph.prebuilt import ToolNode
from typing import Literal
from langchain_core.tools import tool
from pydantic import BaseModel , Field

In [73]:
from google.colab import userdata
api_key=userdata.get('groq_api_key')

In [74]:
llm=ChatGroq(model="openai/gpt-oss-20b",api_key=api_key)

In [75]:
class AgentState(MessagesState):
  document:list[str]

In [88]:
class GradeDocument(BaseModel):
  binary_score:Literal["yes","no"]=Field(description="Choose yes or no based the user question or not")

In [102]:
llm_with_structured_output=llm.with_structured_output(GradeDocument)
print(llm_with_structured_output)

first=RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x7f3c692dc800>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7f3cb6febad0>, model_name='openai/gpt-oss-20b', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'GradeDocument', 'description': '', 'parameters': {'properties': {'binary_score': {'description': 'Choose yes or no based on the document relevant to user question or not', 'enum': ['yes', 'no'], 'type': 'string'}}, 'required': ['binary_score'], 'type': 'object'}}}], 'ls_structured_output_format': {'kwargs': {'method': 'function_calling'}, 'schema': {'type': 'function', 'function': {'nam

In [81]:
def retriever_rag(state:AgentState):
  query=state["messages"][-1]
  docs=retriever.invoke(query)
  return {"document":docs}


In [94]:
from langchain_core.messages import HumanMessage

In [101]:
result=retriever_rag({"messages":"how many hospitals are there in jhapa?"})
formatted_docs=[]
for doc in result["document"]:
  formatted_docs.append(doc.page_content)
llm_with_structured_output.invoke("How many hospitals are there in jhapa?")

BadRequestError: Error code: 400 - {'error': {'message': 'Tool choice is required, but model did not call a tool', 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': 'Jhapa District in Nepal is a fairly well‑served area when it comes to health facilities.  While the exact number can vary depending on how you count “hospital” (some community health centers are sometimes classified as hospitals), the district hosts roughly **10–12 major hospitals** and a handful of smaller health centers and clinics.  \n\n**Major hospitals you’ll find there include:**\n\n| Hospital | Type | Notes |\n|----------|------|-------|\n| **B.P.\u202fKoirala Institute of Health Sciences (BPKIHS)** | Tertiary public | Dharan – one of Nepal’s leading teaching hospitals |\n| **Kanti Hospital** | Private | Dharan – large multi‑specialty facility |\n| **Shree Siddhartha Hospital** | Private | Dharan – well‑known for general and specialist services |\n| **Jhapa District Hospital** | Public